# Problem Set 3 (24 points)

Use the same `sentencing_cleaned` data from Problem Set 2 for this assignment. 

In Problem Set 2, you investigated one form of disparity in the US criminal justice system: probation versus incarceration.

Here, you'll investigate a second type of disparity---the length of a defendant's sentence---and also investigate the disparities faced by defendants sentenced by the same judge for the same crime. 

As a reminder, the codebook is available at this link:  https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf)

# 0. Load packages and imports

In [123]:
## basic functionality
!pip install gender_guesser
!pip install xlrd
!pip install openpyxl

import gender_guesser.detector as gender
import pandas as pd
import numpy as np
import re
import os
import plotnine
from plotnine import *

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 1.1 Filter to defendants who were incarcerated and construct a sentence length variable (10 points)

**Part A:**

- Filter to sentences that involve incarceration (same Illinois Department of Corrections logic as in problem set two: 
incarceration is indicated by `COMMITMENT_TYPE` == "Illinois Department of Corrections")
- Filter out non-numeric sentence lengths (e.g., Term, Pounds, or Dollars)
- Filter to Black or White defendants

**Concepts tested and resources**: this question tests filtering rows based on logical conditions. Here are some resources:
- DataCamp on .loc: https://campus.datacamp.com/courses/data-manipulation-with-pandas/slicing-and-indexing-dataframes?ex=3
- Row subsetting section in this activity: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/01_pandas_datacleaning_solutions.ipynb


### Part A

In [2]:
# read data
case_data = pd.read_csv('sentencing_cleaned.csv')

C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\1112120375.py:2: DtypeWarning: Columns (10,11,16,25) have mixed types. Specify dtype option on import or set low_memory=False.


In [3]:
# Filter out non-numeric sentence lengths
case_data = case_data.loc[~case_data.COMMITMENT_UNIT.isin(['Term', 'Pounds', 'Dollars'])]

In [4]:
# Filter to incarceration and Black or White
case_data_clean = case_data.loc[(case_data.COMMITMENT_TYPE == "Illinois Department of Corrections") & ((case_data.is_black_derived == True) | (case_data.is_white_derived == True))]
case_data_clean

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,simplified_offense_derived,is_black_derived,is_hisp_derived,is_white_derived,is_other_derived,is_male_derived,age_derived,sentenceymd_derived,sentenceym_derived,judgeid_derived
0,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,True,50510062193,112898098217,FIRST DEGREE MURDER,1,12/17/2014 12:00:00 AM,...,Homicide,True,False,False,False,True,27.0,2014-10-16,2014-10-01,judge_40
1,150065796098,162105612284,8/23/1984 12:00:00 AM,PROMIS Conversion,True,50792360681,113332130159,FIRST DEGREE MURDER,1,8/6/2014 12:00:00 AM,...,Homicide,True,False,False,False,True,30.0,2014-08-06,2014-08-01,judge_310
5,155685924389,221749325828,6/15/2001 12:00:00 AM,PROMIS Conversion,True,55017536987,687589180737,AGGRAVATED CRIMINAL SEXUAL ASSAULT,1,5/3/2013 12:00:00 AM,...,PROMIS Conversion,True,False,False,False,True,NaN,2013-05-03,2013-05-01,judge_287
7,156558681704,231553844285,7/17/2001 12:00:00 AM,PROMIS Conversion,True,58235136444,720561595284,2ND DEGREE MURDER,1,12/4/2014 12:00:00 AM,...,Homicide,True,False,False,False,True,17.0,2014-12-04,2014-12-01,judge_329
9,156996749190,238415836990,10/20/2001 12:00:00 AM,PROMIS Conversion,True,58424726399,219609793234,MURDER/INTENT TO KILL/INJURE,1,3/14/2012 12:00:00 AM,...,Homicide,True,False,False,False,True,23.0,2012-03-14,2012-03-01,judge_113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135117,444576672584,902195288394,11/19/2020 12:00:00 AM,Driving With Suspended Or Revoked License,True,446042822953,832902252993,"FELONY DRIVING WHILE DRIVER?S LICENSE, PERMIT,...",1,3/16/2021 12:00:00 AM,...,Driving With Suspended Or Revoked License,True,False,False,False,True,45.0,2021-02-19,2021-02-01,judge_287
135128,444757010121,902797855005,12/4/2020 12:00:00 AM,Credit Card Cases,True,446970360764,834641370221,IDENTITY THEFT,1,3/2/2021 12:00:00 AM,...,Identity Theft,False,False,True,False,True,44.0,2021-02-17,2021-02-01,judge_90
135143,444982872606,903587656097,9/19/2014 12:00:00 AM,Forgery,True,446915459615,834537766815,FORGERY,1,3/19/2021 12:00:00 AM,...,Forgery,True,False,False,False,True,33.0,2021-03-03,2021-03-01,judge_287
135151,445140741379,904087923223,1/7/2021 12:00:00 AM,Possession of Stolen Motor Vehicle,True,447180111306,837593366637,ATTEMPT POSSESSION OF A STOLEN MOTOR VEHICLE,1,3/2/2021 12:00:00 AM,...,Vehicle-related,True,False,False,False,True,24.0,2021-03-02,2021-03-01,judge_4


### Part B


**Part B**: Then, follow the instructions in the codebook (combining `COMMITMENT_TERM` with `COMMITMENT_UNIT`) to create a standard sentence length in days column (`senlength_derived`) that measures the sentence in days. To simplify, you can assume that:

- 1 hour = 1/24th of a day
- 1 year = 365 days
- 1 month = 30.5 days
- 1 week = 7 days
- Natural life = difference between the age of 100 and the defendant's age at incident (cleaned; if missing, code to age 20); note that this is a simplification since age at incident != age at sentencing 

Print the following cols for an example of each type (eg an example of originally hours; an example of natural life): `COMMITMENT_TERM`, `COMMITMENT_UNIT`, `age_derived` and your new standardized sentence length column

Print the summary of that column (`senlength_derived`) using the .describe() command

**Concepts tested and resources**: there are many approaches but a couple ways are:
- np.select covered in the slides and this activity notebook: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/01_pandas_datacleaning_solutions.ipynb
- writing a function that takes in one row as an argument and has a series of if, elif, else conditions where different commitment_units are translated into days. To execute this function, you can use the .apply function but apply it with axis = 1 (row-wise). Resources for that include: (1) the activity notebook on user-defined functions (https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/02_functions_part1_solutions.ipynb); (2) the activity notebook covering apply (same as above)

**Hint on output**: see GitHub issue for the summary stats we get from running .describe()

In [5]:
df = case_data_clean.copy()

df['COMMITMENT_TERM'] = pd.to_numeric(df['COMMITMENT_TERM'], errors='coerce')

df['AGE_AT_INCIDENT'] = df['AGE_AT_INCIDENT'].fillna(20)

conditions_sentence = [df['COMMITMENT_UNIT'] == 'Hours', 
                       df['COMMITMENT_UNIT'] == 'Year(s)', 
                       df['COMMITMENT_UNIT'] == 'Months', 
                     df['COMMITMENT_UNIT'] == 'Weeks', 
                       df['COMMITMENT_UNIT'] == 'Natural Life', 
                       df['COMMITMENT_UNIT'] == 'Days']

code_to = [(1/24)*df['COMMITMENT_TERM'], 
           365*df['COMMITMENT_TERM'], 
           30.5*df['COMMITMENT_TERM'], 
          7*df['COMMITMENT_TERM'], 
           (100-df['AGE_AT_INCIDENT'])*365, 
           1*df['COMMITMENT_TERM']]

df['senlength_derived'] = np.select(conditions_sentence, code_to)

In [6]:
## Your code here on printing example of each type of committment unit and what it's senlength_derived is
h = df.loc[df['COMMITMENT_UNIT']=='Hours'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)
m = df.loc[df['COMMITMENT_UNIT']=='Months'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)
y = df.loc[df['COMMITMENT_UNIT']=='Year(s)'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)
w = df.loc[df['COMMITMENT_UNIT']=='Weeks'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)
d = df.loc[df['COMMITMENT_UNIT']=='Days'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)
n = df.loc[df['COMMITMENT_UNIT']=='Natural Life'][['COMMITMENT_TERM', 'COMMITMENT_UNIT', 'age_derived', 'senlength_derived']].head(1)

pd.concat([h,m,y,w,d,n])

,COMMITMENT_TERM,COMMITMENT_UNIT,age_derived,senlength_derived
92475,1.0,Hours,22.0,0.041667
194,18.0,Months,29.0,549.000000
0,62.0,Year(s),27.0,22630.000000
15310,2.0,Weeks,23.0,14.000000
4697,90.0,Days,40.0,90.000000
1,1.0,Natural Life,30.0,25550.000000


In [7]:
## Your code here with the .describe() command summary of the senlength_derived column
df.senlength_derived.describe()

count     58289.000000
mean       1396.720826
std        2062.874778
min           0.000000
25%         366.000000
50%         915.000000
75%        1460.000000
max      147825.000000
Name: senlength_derived, dtype: float64

## 1.2 Examine disparities in length within the same judge and offense category: constructing matched pairs (14 points)



### Part A 

Keep the above ~58k row dataset subsetting only to sentences involving incarceration. Then, further subset the rows to:
- Those sentenced `judgeid_derived` = `judge_21` 
- `simplified_offense_derived` == "Narcotics"

Use `shape` to print the dimensions of the resulting dataframe

**Concepts and resources**: row subsetting using logical conditions; see above resources
 

In [9]:
## your code here to filter rows and check the shape
case_subset = df.loc[(df.judgeid_derived == 'judge_21') & (df.simplified_offense_derived == 'Narcotics')].copy()

case_subset.shape

(53, 53)

### Part B

For each defendant sentenced by judge_21, you want to construct "matched groups" of defendants who:

- Are the same exact age and
- Are the same gender but 
- Differ in race from the focal defendant

Write a user-defined function to find any/all matched defendants for each focal defendant of judge 21. You can structure the function in various ways but one way is to write a function similar to the class example where we find similar crimes to a focal crime for one focal crime; in this case, we want to:

- Iterate over unique defendants sentenced by judge 21 (use `CASE_PARTICIPANT_ID` to identify each unique defendant)
- Find other defendants in the judge 21 pool who (1) have a different race from that focal defendant but (2) the same gender and age 


**Concepts and resources**: 

- Slides and activity code on user-defined functions and iterating using list comprehension: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/02_functions_part1_solutions.ipynb 
- You can either write code in the function to add columns with the attributes of the focal defendant (existing material) or using `pd.merge` to join these on after; we'll be covering `pd.merge` on Wednesday 09.21 but here are some slides (slide 17-20) from my previous course in meantime: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/s21_slides/qss20_s21_class4.pdf 


**Hints on output**: 

- Some focal defendants may not have any matches; they can be excluded from the results 
- In the way we wrote our function, each iteration of the function returns a single dataframe with the focal defendant's matched defendants


In [13]:
## your code here to define function

def judge_21_sentence (ID : int, dataframe: pd.DataFrame):

#create object group and general group
    focal_def = dataframe[dataframe.CASE_PARTICIPANT_ID == ID].copy()
    focal_def_general = dataframe[dataframe.CASE_PARTICIPANT_ID != ID].copy()

# filter the general group to a match group with same age, gender, different race
    focal_def_match = focal_def_general[(focal_def_general.age_derived.isin(focal_def.age_derived)) &
                       (focal_def_general.GENDER.isin(focal_def.GENDER)) & 
                       (~focal_def_general.RACE.isin(focal_def.RACE))]
    
#create focal_def_match dataframe
    focal_def_match['focal_id'] = ID
    focal_def_match['focal_race'] = focal_def.RACE.iloc[0]
    focal_def_match['focal_def_senlength'] = focal_def.senlength_derived.iloc[0]
    
    return(focal_def_match)

In [17]:
judge_21_sentence(ID = 808109112733, dataframe = case_subset)[['focal_race','focal_id','focal_def_senlength']]

C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,focal_race,focal_id,focal_def_senlength
47992,Black,808109112733,2190.0
49418,Black,808109112733,2190.0
53700,Black,808109112733,2190.0


**Part B**: using the results from Part A, use `pd.concat` or another approach to create a dataframe that compares the (1) race and sentence length for the focal defendant to (2) the sentence length for other defendants. Using this dataframe, show this comparison for focal defendant: `CASE_PARTICIPANT_ID` == `808109112733`


In [22]:
## your code to rowbind all matches and to highlight the comparison

IDs = list(case_subset.CASE_PARTICIPANT_ID.unique())

match_IDs = [judge_21_sentence (ID, case_subset) for ID in IDs]

match_data = pd.concat(match_IDs).drop_duplicates()

C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuyuan\AppData\Local\Temp\ipykernel_2052\2347062517.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [21]:
match_data.loc[match_data['focal_id'] == 808109112733][['CASE_PARTICIPANT_ID', 'RACE', 'focal_race','focal_id','focal_def_senlength','senlength_derived']]

,CASE_PARTICIPANT_ID,RACE,focal_race,focal_id,focal_def_senlength,senlength_derived
47992,768307912970,White,Black,808109112733,2190.0,730.0
49418,769939231128,White,Black,808109112733,2190.0,730.0
53700,774967571640,White,Black,808109112733,2190.0,365.0


**Part C**: group by the focal defendant's race and find the proportion of that defendant's matches who had a LONGER sentence than the focal defendant

**Concepts and resources**: can use groupby and agg

- Groupby and agg code: https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/01_pandas_datacleaning_solutions.ipynb

In [65]:
match_data['select'] = match_data.senlength_derived > match_data.focal_def_senlength
match_data.groupby('focal_race').agg(sentence_longer_prop = ('select', np.mean))

,sentence_longer_prop
focal_race,
Black,0.483871
White,0.451613


**Part D**: write 1-2 lines commenting on the results from Part C. What other defendant or offense-level characteristics would you like to match on to investigate claims about racial disparities? 


The result shows that for white defendents, thier (black) match group's sentences is longer 45% of the time. For black defendents, thier (white) match group's sentences is longer 48% of the time. I think other characteristics can be judge's race and sentenceymd_derived. 

# 2. Optional extra credit/challenge exercise (1 point): impute possible judge demographic correlates of sentencing

The previous exercises showed large differences in sentences between judges/differences in disparities. You become interested in how the judge's own demographic attributes are correlated with sentencing. Going back to the judge's name (`SENTENCE JUDGE`), parse their first name and try to probabilistically infer his or her gender. Then, investigate whether disparities differ between "likely female" and "likely male" judges. 

**Note on ethics of probabilistic inference of attributes based on name**: Using names to infer demographic characteristics has become increasingly popular with the rise of "digital trace data" that often lacks explicit demographic fields (e.g., tweets just have usernames and profiles; academic citation networks just have author names; voter registration records sometimes just have names and not race/ethnicity). But there are many valid ethical critiques of this practice. In the case of gender, a person's assigned name at birth clearly does not always map onto their self-identified gender, both due to gender fluidity and names like "Morgan." A couple critiques I link to are:

- [This blog post](https://scatter.wordpress.com/2021/07/30/who-writes-social-science/)
- [Urban Institute ethical risks of race/ethnicity imputation - applies to gender](https://www.urban.org/research/publication/five-ethical-risks-consider-filling-missing-race-and-ethnicity-data)

 

In [95]:
# insert code here
df['SENTENCE_JUDGE']

0          Clayton Jay Crane
1            Thomas V Gainer
5            Steven J Goebel
7            William H Hooks
9             James L Rhodes
                 ...        
135117       Steven J Goebel
135128          Geary W Kull
135143       Steven J Goebel
135151    Alfredo  Maldonado
135158       Joseph  Cataldo
Name: SENTENCE_JUDGE, Length: 58289, dtype: object

In [97]:
df['first_name'] = df['SENTENCE_JUDGE'].str.split().str[0]
df['first_name']

0         Clayton
1          Thomas
5          Steven
7         William
9           James
           ...   
135117     Steven
135128      Geary
135143     Steven
135151    Alfredo
135158     Joseph
Name: first_name, Length: 58289, dtype: object

In [140]:
# code reference comes from https://stackoverflow.com/questions/68934064/identify-gender-from-the-first-name-python
gd = gender.Detector()
df['gender'] = df['first_name'].apply(str.capitalize).map(gd.get_gender)
df[['first_name', 'gender']]

,first_name,gender
0,Clayton,male
1,Thomas,male
5,Steven,male
7,William,male
9,James,male
...,...,...
135117,Steven,male
135128,Geary,male
135143,Steven,male
135151,Alfredo,male


In [139]:
df['gender'].value_counts()

male             38494
female           13928
mostly_female     4232
unknown           1583
andy                51
mostly_male          1
Name: gender, dtype: int64

In [145]:
df.groupby('gender').agg(sentence_mean = ('senlength_derived', 'mean'))

,sentence_mean
gender,
andy,869.705882
female,1366.990681
male,1426.683666
mostly_female,1260.559546
mostly_male,1095.000000
unknown,1310.874921


Based on the result, we found that the mean of sentence length for mostly_female is longer than mostlt_male, this doesn't imply there is a disparity between likely men and likely women. We need more detail data to examine if there is a disparity. 

# 3. Optional extra credit/challenge exercise (1 point): examine disparities across multiple judges

In problem 1.2, we focused on one judge: judge 21
    
For this extra credit exercise:
    
- Get judges with a sufficient sample size: filter to each judge with at least 20 Black and at least 20 white defendants
- Calculate the within-judge disparity and plot the (1) median sentence length for Black defendants and (2) median sentence length for white defendants (factor variable on x axis for each judge_id who qualifies; group by race)
- Write a 1-2 sentence interpretation - if we assume that cases/defendants are randomly assigned to sentencing judges, what might this suggest about the role of judicial discretion in these disparities?


In [121]:
# insert code here
sample = df.groupby('judgeid_derived').agg(Black = ('is_black_derived', 'sum'),
                                        White = ('is_white_derived', 'sum')).reset_index()
sample.loc[(sample.Black >= 20) & (sample.White >= 20)]

,judgeid_derived,Black,White
1,judge_10,311,32
2,judge_100,360,122
6,judge_109,1100,62
10,judge_115,1184,67
11,judge_116,120,165
...,...,...,...
194,judge_81,516,36
196,judge_83,599,41
197,judge_87,509,60
198,judge_89,179,128


In [177]:
df.groupby(['judgeid_derived', 'is_white_derived']).agg(median_sen = ('senlength_derived', np.median))

median_sen
judgeid_derived is_white_derived            
judge_1         False                 730.00
                True                  365.00
judge_10        False                1281.00
                True                  730.00
judge_100       False                1095.00
...                                      ...
judge_95        False                 365.00
judge_98        False                 638.75
                True                  365.00
judge_99        False                1552.50
                True                  365.00

[345 rows x 1 columns]